In [80]:
import pandas as pd


In [81]:
data = pd.read_csv('Task_1\images\metadata.csv')
data.head()

C:\Users\yannn\AppData\Local\Temp\ipykernel_29436\1000669844.py:1: DtypeWarning: Columns (8,11,13,14,15,17,18,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Task_1\images\metadata.csv')


,isic_id,attribution,copyright_license,acquisition_day,age_approx,anatom_site_general,benign_malignant,clin_size_long_diam_mm,dermoscopic_type,diagnosis,...,mel_thick_mm,mel_type,mel_ulcer,melanocytic,nevus_type,patient_id,personal_hx_mm,pixels_x,pixels_y,sex
0,ISIC_3491559,Memorial Sloan Kettering Cancer Center,CC-0,NaN,45.0,upper extremity,benign,1.4,NaN,seborrheic keratosis,...,NaN,NaN,NaN,True,NaN,IP_0918668,False,3264,2448,female
1,ISIC_3622379,Memorial Sloan Kettering Cancer Center,CC-0,NaN,65.0,posterior torso,benign,8.8,contact polarized,nevus,...,NaN,NaN,NaN,True,NaN,IP_6028537,True,3264,2448,male
2,ISIC_2443008,Memorial Sloan Kettering Cancer Center,CC-0,NaN,45.0,upper extremity,malignant,7.7,non-contact polarized,squamous cell carcinoma,...,NaN,NaN,NaN,False,NaN,IP_2891227,False,3264,2448,male
3,ISIC_6892286,Memorial Sloan Kettering Cancer Center,CC-0,NaN,75.0,posterior torso,malignant,9.4,contact polarized,melanoma,...,0.3,NaN,False,True,NaN,IP_3056186,True,3264,2448,male
4,ISIC_1979534,Memorial Sloan Kettering Cancer Center,CC-0,NaN,80.0,head/neck,malignant,20.0,NaN,squamous cell carcinoma,...,NaN,NaN,NaN,False,NaN,IP_2084621,True,3264,2448,male


In [82]:
columns = data.columns.tolist()

In [83]:
print(columns)

['isic_id', 'attribution', 'copyright_license', 'acquisition_day', 'age_approx', 'anatom_site_general', 'benign_malignant', 'clin_size_long_diam_mm', 'dermoscopic_type', 'diagnosis', 'diagnosis_confirm_type', 'family_hx_mm', 'image_type', 'lesion_id', 'mel_class', 'mel_mitotic_index', 'mel_thick_mm', 'mel_type', 'mel_ulcer', 'melanocytic', 'nevus_type', 'patient_id', 'personal_hx_mm', 'pixels_x', 'pixels_y', 'sex']


In [128]:
df = data[['isic_id', 'diagnosis', 'benign_malignant']]
df.head()

,isic_id,diagnosis,benign_malignant
0,ISIC_3491559,seborrheic keratosis,benign
1,ISIC_3622379,nevus,benign
2,ISIC_2443008,squamous cell carcinoma,malignant
3,ISIC_6892286,melanoma,malignant
4,ISIC_1979534,squamous cell carcinoma,malignant


In [129]:
value_counts = df.diagnosis.value_counts()
value_counts

nevus                                 29091
melanoma                               5953
basal cell carcinoma                   3417
seborrheic keratosis                   1562
pigmented benign keratosis             1100
actinic keratosis                       929
squamous cell carcinoma                 688
solar lentigo                           428
dermatofibroma                          264
vascular lesion                         259
lentigo NOS                             222
atypical melanocytic proliferation       99
lichenoid keratosis                      91
lentigo simplex                          31
other                                    25
AIMP                                     22
angioma                                  16
neurofibroma                              7
scar                                      6
verruca                                   6
acrochordon                               5
melanoma metastasis                       4
angiofibroma or fibrous papule  

In [131]:
for group_name, group in df.groupby('diagnosis'):
    if len(group) < 1000:
        df.loc[df['diagnosis'] == group_name, 'diagnosis'] = group['benign_malignant'].values
        
df.head(25)

C:\Users\yannn\AppData\Local\Temp\ipykernel_29436\2135353932.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['diagnosis'] == group_name, 'diagnosis'] = group['benign_malignant'].values


,isic_id,diagnosis,benign_malignant
0,ISIC_3491559,seborrheic keratosis,benign
1,ISIC_3622379,nevus,benign
2,ISIC_2443008,malignant,malignant
3,ISIC_6892286,melanoma,malignant
4,ISIC_1979534,malignant,malignant
5,ISIC_1033970,nevus,benign
6,ISIC_3678711,benign,benign
7,ISIC_3954972,seborrheic keratosis,benign
8,ISIC_9214649,melanoma,malignant
9,ISIC_6688399,nevus,benign


In [132]:
df.diagnosis.value_counts()

nevus                         29091
melanoma                       5953
basal cell carcinoma           3417
seborrheic keratosis           1562
pigmented benign keratosis     1100
benign                          751
indeterminate                    71
malignant                        57
indeterminate/benign             24
indeterminate/malignant           2
Name: diagnosis, dtype: int64

In [134]:
n_samples = 751
# Group the data by a categorical variable
grouped = df.groupby('diagnosis')

# Filter out groups that have less than n_samples
grouped_filtered = grouped.filter(lambda x: len(x) >= n_samples)

# Sample the same number of items from each group
subset = grouped_filtered.groupby('diagnosis').apply(lambda x: x.sample(n=n_samples))

# Reset the index of the resulting subset
subset = subset.reset_index(drop=True)

In [135]:
subset.head()

,isic_id,diagnosis,benign_malignant
0,ISIC_0073221,basal cell carcinoma,NaN
1,ISIC_0059877,basal cell carcinoma,NaN
2,ISIC_0069975,basal cell carcinoma,NaN
3,ISIC_0071345,basal cell carcinoma,NaN
4,ISIC_0065739,basal cell carcinoma,NaN


In [137]:
subset.diagnosis.value_counts()

basal cell carcinoma          751
benign                        751
melanoma                      751
nevus                         751
pigmented benign keratosis    751
seborrheic keratosis          751
Name: diagnosis, dtype: int64

In [138]:
images = [name + '.jpg' for name in subset.isic_id.tolist()]
labels = subset.diagnosis.tolist()

In [139]:
from sklearn import model_selection, preprocessing

In [140]:
le = preprocessing.LabelEncoder()
encoded = le.fit_transform(labels)
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{'basal cell carcinoma': 0, 'benign': 1, 'melanoma': 2, 'nevus': 3, 'pigmented benign keratosis': 4, 'seborrheic keratosis': 5}


In [70]:
trainx, testx, trainy, testy = model_selection.train_test_split(images, encoded, test_size=.3)

In [71]:
trainx, valx, trainy, valy = model_selection.train_test_split(trainx, trainy, train_size=.8)

In [74]:
def make_csv(images, labels, name):
    df = pd.DataFrame([[image, label] for image, label in zip(images, labels) 
                       ], columns=['images', 'labels'])
    df.to_csv(f'{name}.csv', index=False)

In [75]:
make_csv(trainx,trainy,'subset/train')
make_csv(valx,valy,'subset/val')
make_csv(testx,testy,'subset/test')

In [79]:
!python imagenet.py --arch mobilenet_v3_small --accelerator gpu --auto_scale_batch_size --precision 16 --amp_backend apex --amp_level 'O3' --data-path Task_1/images/ --pretrained 

usage: imagenet.py [--logger [LOGGER]]
                   [--enable_checkpointing [ENABLE_CHECKPOINTING]]
                   [--default_root_dir DEFAULT_ROOT_DIR]
                   [--gradient_clip_val GRADIENT_CLIP_VAL]
                   [--gradient_clip_algorithm GRADIENT_CLIP_ALGORITHM]
                   [--num_nodes NUM_NODES] [--num_processes NUM_PROCESSES]
                   [--devices DEVICES] [--gpus GPUS]
                   [--auto_select_gpus [AUTO_SELECT_GPUS]]
                   [--tpu_cores TPU_CORES] [--ipus IPUS]
                   [--enable_progress_bar [ENABLE_PROGRESS_BAR]]
                   [--overfit_batches OVERFIT_BATCHES]
                   [--track_grad_norm TRACK_GRAD_NORM]
                   [--check_val_every_n_epoch CHECK_VAL_EVERY_N_EPOCH]
                   [--fast_dev_run [FAST_DEV_RUN]]
                   [--accumulate_grad_batches ACCUMULATE_GRAD_BATCHES]
                   [--max_epochs MAX_EPOCHS] [--min_epochs MIN_EPOCHS]
                   [--ma